### 지니 Top200 차트

In [59]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

##### 1. 데이터가져오기

In [60]:
url = 'https://www.genie.co.kr/chart/top200'
result = requests.get(url)
result.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [61]:
# Chrome User-Agent
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')

##### 2. 원하는 데이터 찾기

In [62]:
trs = soup.select('tr.list')
len(trs)

50

In [63]:
# 개발자 도구에서 찾아주는 Selector
# #body-content > div.newest-list > div > table > tbody > tr:nth-child(1)
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

##### 3. 하나를 선택해서 원하는 정보 찾기

In [64]:
tr = trs[0]

In [65]:
rank = tr.select_one('.number').get_text()
rank

'1\n                                        \n                                    \n유지\n\n'

In [66]:
# rank
rank = tr.select_one('.number').get_text().split('\n')[0].strip()
rank

'1'

In [67]:
# img
img = 'https:' + tr.select_one('.cover > img')['src']
img

'https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/559/83665559_1681116738103_1_140x140.JPG/dims/resize/Q_80,0'

In [68]:
# title
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'I AM'

In [69]:
# 19금 노래 제목
title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
title

'I AM'

In [70]:
# 위의 두 코드를 합치면
# 방법 1
title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

# 방법 2
try:
    title = tr.select_one('.title.ellipsis').get_text().strip()
except:
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()


In [71]:
# artist
artist = tr.select_one('.artist.ellipsis').string.strip()
artist

'IVE (아이브)'

In [72]:
# album
album = tr.select_one('.albumtitle.ellipsis').text.strip()
album

"I've IVE"

##### 4. 한 페이지내에 있는 정보

In [73]:
# 방법 1. 딕셔너리의 리스트로 가져오기
line = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0].strip()
    img = 'https:' + tr.select_one('.cover > img')['src']
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').string.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    line.append({'rank':rank, 'title':title, 'artist': artist, 'album': album, 'img': img})

df = pd.DataFrame(line)
df.head()


,rank,title,artist,album,img
0,1,I AM,IVE (아이브),I've IVE,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
1,2,Kitsch,IVE (아이브),I've IVE,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
2,3,꽃,지수 (JISOO),ME,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
3,4,Ditto,NewJeans,NewJeans 'OMG',https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
4,5,CHRISTIAN,Zior Park,WHERE DOES SASQUATCH LIVE? PART 1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...


In [74]:
# 방법 2. 개별 리스트로 가져오기
rank_list, img_list, title_list, artist_list, album_list = [], [], [], [], []
for tr in trs:
    rank_list.append(tr.select_one('.number').get_text().split('\n')[0].strip())
    img_list.append('https:' + tr.select_one('.cover > img')['src'])
    title_list.append(tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip())
    artist_list.append(tr.select_one('.artist.ellipsis').string.strip())
    album_list.append(tr.select_one('.albumtitle.ellipsis').text.strip())

df2 = pd.DataFrame({
    'rank':rank_list, 'title': title_list, 'artist': artist_list, 'album' : album_list, 'img':img_list
})
df2.head()

,rank,title,artist,album,img
0,1,I AM,IVE (아이브),I've IVE,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
1,2,Kitsch,IVE (아이브),I've IVE,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
2,3,꽃,지수 (JISOO),ME,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
3,4,Ditto,NewJeans,NewJeans 'OMG',https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
4,5,CHRISTIAN,Zior Park,WHERE DOES SASQUATCH LIVE? PART 1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...


##### 5. 에러 처리

##### 6. 모든 페이지에 대해서 처리하기

In [75]:
# 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20230424&hh=15&rtm=Y&pg='
from datetime import datetime
now = datetime.now()
now

datetime.datetime(2023, 4, 24, 15, 25, 31, 840566)

In [76]:
ymd = now.strftime('%Y%m%d')    # ymd = f'{now.year}{now.month:02d}{now.day:02d}'
hh = now.strftime('%H')         # hh = f'{now.hour:02d}'

ymd, hh

('20230424', '15')

In [77]:
from tqdm import tqdm

lines = []
base_url = 'https://www.genie.co.kr/chart/top200?ditc=D'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

for page in tqdm(range(1, 5)):
    url = f'{base_url}&ymd={ymd}&hh={hh}&rtm=Y&pg={page}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')
    trs = soup.select('tr.list')

    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0].strip()
        img = 'https:' + tr.select_one('.cover > img')['src']
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
        artist = tr.select_one('.artist.ellipsis').string.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
        lines.append({'rank':rank, 'title':title, 'artist': artist, 'album': album, 'img': img})


100%|██████████| 4/4 [00:04<00:00,  1.04s/it]


##### 7. 데이터프레임 만들기

In [78]:
df3 = pd.DataFrame(lines)
df3.head()

,rank,title,artist,album,img
0,1,I AM,IVE (아이브),I've IVE,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
1,2,Kitsch,IVE (아이브),I've IVE,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
2,3,꽃,지수 (JISOO),ME,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
3,4,Ditto,NewJeans,NewJeans 'OMG',https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...
4,5,CHRISTIAN,Zior Park,WHERE DOES SASQUATCH LIVE? PART 1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...


In [79]:
df3.to_csv(f'data/{ymd}{hh}_지니차트.csv', index=False)